# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool


import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope_s(y_axis):
    global window_s_ma
    x_axis = []
    for i in range(window_s_ma):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

def get_slope_l(y_axis):
    global window_l_ma
    x_axis = []
    for i in range(window_l_ma):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

In [3]:
def chunk_ticks(df, number_of_ticks):   
    global pip_diff
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    tick_avg = []
    spread_avg = []
    tick_sd = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])
        
        tick_avg.append(np.mean(tick_list))
        spread_avg.append(np.mean(spread_list))
        tick_sd.append(np.std(tick_list))
        
        
    temp_df['tick_avg'] = tick_avg  
    temp_df['spread_avg'] = spread_avg  
    temp_df['tick_sd'] = tick_sd  
    
    return(temp_df)

## Parameters

In [4]:
year = 2018
source_file_path = f'data/tick_{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')


number_of_ticks = 10
pip_diff = 0.00001
rsi_window = 14
rs_max = 1e6
window_s_ma = 10
window_l_ma = 100

source_file_path : data/tick_2018.csv
chunk_file_path : data\chunk_tick_2018.csv
target_file_path : data\tab_tick_2018.csv


## Read data

In [5]:
%%time
df = pd.read_csv(source_file_path, nrows=1000000)
#df = pd.read_csv(source_file_path)
df.head()

Wall time: 863 ms


,DateTime,Bid,Ask,Volume
0,20180101 22:00:08.661,1.20102,1.20143,2
1,20180101 22:00:08.895,1.20102,1.20148,2
2,20180101 22:00:10.634,1.20102,1.20147,2
3,20180101 22:00:11.223,1.20102,1.20148,2
4,20180101 22:00:29.530,1.20102,1.20145,2


## Data manipulation

In [6]:
%%time
df = chunk_ticks(df, number_of_ticks)

df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')
df = pd.read_csv(chunk_file_path)
df.head()

100%|██████████████████████████████████████| 100000/100000 [00:36<00:00, 2773.76it/s]


Records : 100000
Wall time: 37.1 s


,tick_avg,spread_avg,tick_sd
0,1.201194,0.000368,0.000073
1,1.200771,0.000397,0.000157
2,1.200578,0.000756,0.000054
3,1.200736,0.000652,0.000038
4,1.200714,0.000800,0.000020


In [7]:
%%time
df['tick_diff'] = df['tick_avg'].diff()

# Moving Averages ------------------
df['sema'] = df['tick_avg'].ewm(span=window_s_ma).mean()
df['lema'] = df['tick_avg'].ewm(span=window_l_ma).mean()
df['ema_diff'] = df['sema'] - df['lema']

df['ssma'] = df['tick_avg'].rolling(window=window_s_ma).mean()
df['lsma'] = df['tick_avg'].rolling(window=window_l_ma).mean()
df['sma_diff'] = df['ssma'] - df['lsma']

# RSI -----------------------------
df['diff'] = df['sema'].diff()
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])
df['avg_gain'] = df['gain'].rolling(window=rsi_window).mean()
df['avg_loss'] = df['loss'].rolling(window=rsi_window).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= rs_max, rs_max) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

# Slopes -----------------------------
df['small_sema_slope'] = df['sema'].rolling(window=window_s_ma).progress_apply(get_slope_s)
df['long_sema_slope'] = df['sema'].rolling(window=window_l_ma).progress_apply(get_slope_l)

# Direction -------------------------
diff_col = 'sema'
df['direction'] = 'same'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] > pip_diff] = 'increase'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -pip_diff] = 'decrease'

# Remove NaNs ------------------------
df = df.dropna()
df = df.reset_index(drop=True)
print(f'Total records : {len(df)}')

99991it [00:49, 2034.64it/s]
99901it [00:58, 1718.36it/s]


Total records : 99901
Wall time: 1min 47s


## Write data to csv

In [8]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 4.46 s


## Print Report

In [9]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head(5)

          counts  percentage
same       48793   48.841353
increase   25620   25.645389
decrease   25488   25.513258


,tick_avg,spread_avg,tick_sd,tick_diff,sema,lema,ema_diff,ssma,lsma,sma_diff,diff,gain,loss,avg_gain,avg_loss,rs,rsi,small_sema_slope,long_sema_slope,direction
0,1.201482,0.000157,0.000007,1.000000e-05,1.201457,1.201171,0.000286,1.201439,1.201024,0.000414,0.000006,0.000006,0.0,0.000003,0.000005,0.620149,38.277283,-24.468222,36.156279,same
1,1.201481,0.000178,0.000002,-5.000000e-07,1.201461,1.201178,0.000283,1.201441,1.201027,0.000413,0.000004,0.000004,0.0,0.000003,0.000005,0.639789,39.016561,-11.820742,37.596694,same
2,1.201483,0.000176,0.000002,2.000000e-06,1.201465,1.201185,0.000280,1.201448,1.201034,0.000414,0.000004,0.000004,0.0,0.000003,0.000005,0.638914,38.983976,-0.274442,38.488405,same
3,1.201496,0.000173,0.000010,1.350000e-05,1.201471,1.201192,0.000279,1.201455,1.201044,0.000412,0.000006,0.000006,0.0,0.000003,0.000005,0.662203,39.838872,10.405255,39.028280,same
4,1.201492,0.000158,0.000010,-4.500000e-06,1.201475,1.201199,0.000276,1.201462,1.201051,0.000411,0.000004,0.000004,0.0,0.000002,0.000005,0.540048,35.066958,18.440327,39.492326,same
